# 📝 Learning goals of practical

- You can describe how to apply hierarchical clustering to a transcriptomics dataset

- You can discuss the goals of unsupervised machine learning when applied to transcriptomics data

In this practical, we will explore RNA-seq data from the study:

*High-Throughput RNA Sequencing of Pseudomonas-Infected Arabidopsis Reveals Hidden Transcriptome Complexity and Novel Splice Variants* by [Howard et al. (2013)](https://doi.org/10.1371/journal.pone.0074183)

### ❓Questions
Have a look at the paper.

- What three treatments are the plants subjected to?

- What is the goal of these three treatments?

# Hierarchical Clustering

Let's perform hierarchical clustering on this dataset.

In [ ]:
# These cells set up everything

%pip install -q observable_jupyter==0.1.10 clustergrammer2 fastcluster

import sys
if "google.colab" in sys.modules:
    %pip install git+https://github.com/CropXR/EduXR.git
    from google.colab import files
else:
    %load_ext autoreload
    %autoreload 2

from clustergrammer2 import net
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from observable_jupyter import embed
import pandas as pd
from IPython.display import Javascript
from pyvis.network import Network
import networkx as nx

from IPython.core.display import display, HTML
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

!wget https://raw.githubusercontent.com/CropXR/EduXR/refs/heads/main/data/biotic_transcriptomics.txt
net.load_file('biotic_transcriptomics.txt')
net.cluster(dist_type='correlation', linkage_type='average')

Now we can display a heatmap of the samples and their (normalised) gene expressions:

In [ ]:
plt.figure(figsize=(10, 10))
df = pd.read_csv('biotic_transcriptomics.txt', sep='\t', header=[0], index_col=0, skiprows=[1,2])
g= sns.clustermap(df, metric='correlation', method='average', cmap="vlag", vmin=-2, vmax=2)
g.ax_heatmap.set_yticks([])
g.ax_heatmap.set_yticklabels([])
plt.show()

### ❓Questions

- Is the avirulent sample more similar to the virulent sample or the mock sample?

- Does the treatment or the time point play a bigger role for clustering the samples?

- Could you explain the clustering? What does it tell you about the relation between the samples?

We can also show you an interactive plot of this. Adjust the sliders on the right and bottom to find the clustering cutoff. By clicking on the trapezoid that belongs to a cluster you can select the group of genes.

In [ ]:
embed('@cornhundred/clustergrammer-gl', cells=['clustergrammer'],  inputs={'network': net.viz})

### ❓Questions
- How many gene clusters do you think is most appropriate for this dataset? Why?

- Change the clustering parameters, what differences do you observe?

- What group(s) of genes would be most interesting to study?


If you have extra time you can further investigate through what biological process the genes you found are important for stress response, for example by looking up information about them in  databases such as [UniProt](https://www.uniprot.org/). Can you link the genes to certain processes? How does that relate to what you find in literature about this response?

# Correlation network

In [ ]:
def plot_correlation_network(df: pd.DataFrame, threshold: float = 0.9, colour_by=None, interactive=False):
    """_summary_

    Args:
        df (pd.DataFrame): Gene expression dataframe
        threshold (float, optional): Cutoff above which to draw edge between nodes. Defaults to 0.9.
        colour_by (str, optional): 'degree', 'betweenness', or 'closeness'. Defaults to None.
    """
    colour_by_to_func = {'degree': nx.degree_centrality, 'betweenness': nx.betweenness_centrality, 'closeness': nx.closeness_centrality}
    # 1. Compute correlation matrix
    corr_matrix = df.T.corr(method="pearson") 

    # 2. Threshold correlations (you don’t want every edge, it’d be spaghetti)
    edges = [
        (gene1, gene2, corr_matrix.loc[gene1, gene2])
        for gene1 in corr_matrix.index
        for gene2 in corr_matrix.columns
        if gene1 < gene2 and corr_matrix.loc[gene1, gene2] > threshold
    ]

    # 3. Build network
    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    if colour_by:
        node_color_metric = colour_by_to_func[colour_by](G)
        # Print top 10 genes by this metric
        top10 = sorted(node_color_metric.items(), key=lambda x: x[1], reverse=True)[:10]
        print("Top 10 genes by centrality:")
        for gene, value in top10:
            print(f"{gene}: {value:.4f}")

        # Normalise colours for plotting
        values = [node_color_metric[node] for node in G.nodes()]
    else:
        values = "#1f78b4"

    if not interactive:
        # 4. Plot
        plt.figure(figsize=(10, 8))
        pos = nx.kamada_kawai_layout(G)
        nx.draw_networkx_nodes(G, pos, node_size=10, node_color=values, cmap=plt.cm.viridis)
        nx.draw_networkx_edges(G, pos, alpha=0.1)
        # nx.draw_networkx_labels(G, pos, font_size=6)
        plt.title("Gene Correlation Network")
        plt.axis("off")
        plt.show()
    else:
        # Create pyvis network
        net = Network(notebook=True, cdn_resources='in_line', select_menu=True)
        norm = mcolors.Normalize(vmin=min(values), vmax=max(values))
        cmap = cm.get_cmap("viridis")
        # Add nodes with colormap
        for node, centrality in node_color_metric.items():
            rgba = cmap(norm(centrality))
            hex_color = mcolors.to_hex(rgba)
            net.add_node(
                node,
                title=f"{node}<br>Centrality: {centrality:.4f}",
                value=centrality*10,  # adjust size if needed
                color=hex_color,
            )

        # Add edges
        for u, v in G.edges():
            net.add_edge(u, v)

        net.set_options(
        """
        {
        "physics": {
            "forceAtlas2Based": {
            "theta": 0.9,
            "gravitationalConstant": -10,
            "springLength": 5,
            "springConstant": 0.05,
            "damping": 0.7
            },
            "maxVelocity": 20,
            "minVelocity": 0.75,
            "solver": "forceAtlas2Based",
            "timestep": 0.5
        }
        }
        """
        )
        net.show("nx.html")

        display(HTML('nx.html'))
        
plot_correlation_network(df, threshold=0.9)

### ❓Questions
- What happens as you change the correlation threshold?
- Is there a relationship between the hierarchical clustering shown above and the tightly connected nodes you find in this network?

Now explore some node metrics to identify genes that could be worth studying. You can replace `degree` by `betweenness`, or `closeness` to colour nodes based on other properties. How would you interpret these metrics biologically?

In [ ]:
plot_correlation_network(df, threshold=0.9, colour_by='degree', interactive=True)